# Comment section scraper

## Part 1 Basic steps
In the first part we will start by importing the right packages and load the data required to let the scraper work

#### Step 1
First we import the packages required for the scraper to work

In [1]:
import time
import selenium.webdriver
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
from tqdm import tqdm
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

#### Step 2 
First we import the first 200 video codes which will be used to make the scraper work. 

In [24]:
dataset_videos1 = pd.read_csv(r'videos_p1.csv', low_memory = False, sep = ",")
dataset_videos1

,video_id
0,-BCT3VDlkQc
1,-I0FjdwcSm0
2,-M_r8MKQ3mo
3,-Ot0vHUyEpw
4,-r5jmwTFb6M
...,...
195,ydn7Kku1cPg
196,zHytqDhPVgE
197,zisuhZqTeH4
198,zqllxbPWKNI


In [25]:
dataset_videos1 = dataset_videos1['video_id']
dataset_videos1

0      -BCT3VDlkQc
1      -I0FjdwcSm0
2      -M_r8MKQ3mo
3      -Ot0vHUyEpw
4      -r5jmwTFb6M
          ...     
195    ydn7Kku1cPg
196    zHytqDhPVgE
197    zisuhZqTeH4
198    zqllxbPWKNI
199    zy8vIiE8xXY
Name: video_id, Length: 200, dtype: object

## Part 2 Scraper comment section
In this scraper we start to extract the people who makes comments on Youtube and those who react to is. The main goal of this is to extract a dataset per video to detect to what extend the creator of the video reacts to the comments. 

#### Step 3
This scraper will check for every video whether the creator interacted with the audience or not. 
This scraper will work for the first 200 videos.

In [27]:
def interact_creator(urls):
    interactivity = []
    for url in tqdm(urls):
        driver = selenium.webdriver.Chrome()
        driver.get("https://www.youtube.com/watch?v=" + url)

        time.sleep(5)
        consent_button_xpath = "//tp-yt-paper-button[@aria-label='Akkoord gaan met het gebruik van cookies en andere gegevens voor de beschreven doeleinden']"
        consent = driver.find_element_by_xpath(consent_button_xpath)
        consent.click()
    
        driver.execute_script("window.scrollTo(0, 480)") 
        sleep(2.5)
    
        #Load all comments
        while True:
            scroll_height = 5000
            document_height_before = driver.execute_script("return document.documentElement.scrollHeight")
            driver.execute_script(f"window.scrollTo(0, {document_height_before + scroll_height});")
            time.sleep(2)
            document_height_after = driver.execute_script("return document.documentElement.scrollHeight")
            if document_height_after == document_height_before:
                break
        
        res = driver.page_source.encode('utf-8')
        soup = BeautifulSoup(res, "html.parser")
        
        def video_code():
            try:
                video_code = soup.find("meta", itemprop="videoId")["content"]
                return(video_code)
            except:
                return
        
        def hearts():
            try:
                hearts = len(driver.find_elements_by_id('creator-heart-button'))
                return(hearts)
            except:
                return 
            
        def comments():
            try:
                comments = len(driver.find_elements_by_xpath("//yt-formatted-string[@id='text']/span[4]"))
                return(comments)
            except:
                return 
        
        interactivity.append({"video_code" : video_code(),
                                 "hearts" : hearts(),
                                    "comments" : comments()})
        
        sleep (1)
    return interactivity 

In [ ]:
interactive1 = interact_creator(dataset_videos1) 
interactive1

In [6]:
interactivity_creator = pd.DataFrame(interactive)
interactivity_creator.to_csv("interactivity_creator.csv", index=False)
interactivity_creator

,video_code,hearts,comments
0,b8HVQtIoBYU,22,2
1,2tMwq5lHbWg,0,0
2,949C_lwEyFM,0,1
